In [9]:
import numpy as np
from netCDF4 import Dataset
import struct

def read_gen_file_binary(filename):
    with open(filename, 'rb') as file:
        # Read header if necessary
        header = file.read(16)  # Adjust based on actual header size
        # Read the number of elements (example)
        num_elements = struct.unpack('I', header[:4])[0]  # Adjust based on actual header format
        
        # Read the rest of the file as binary data
        data = file.read()
    
    # Check the length of the data
    data_size = len(data)
    print(f"Data size: {data_size} bytes")
    
    # Calculate number of elements from data size
    calculated_num_elements = data_size // (3 * np.dtype(np.float32).itemsize)
    print(f"Calculated num_elements: {calculated_num_elements}")
    
    if calculated_num_elements != num_elements:
        print(f"Warning: Calculated number of elements does not match header value.")
    
    # Process data
    coordinates = np.frombuffer(data, dtype=np.float32).reshape((calculated_num_elements, 3))
    
    return calculated_num_elements, coordinates

def write_ncdf_file(filename, num_elements, coordinates):
    with Dataset(filename, 'w', format='NETCDF4') as ncfile:
        ncfile.createDimension('num_elements', num_elements)
        ncfile.createDimension('num_coords', coordinates.shape[1])

        coords_var = ncfile.createVariable('coordinates', 'f4', ('num_elements', 'num_coords'))
        coords_var[:] = coordinates

        ncfile.description = 'Converted data from .gen to .ncdf'
        # Add additional variables and attributes as needed

        print(f"NetCDF file '{filename}' created successfully.")

# Example usage
num_elements, coordinates = read_gen_file_binary('pillbox-rtop.gen')
if num_elements is not None and coordinates is not None:
    write_ncdf_file('pillbox-rtop.ncdf', num_elements, coordinates)

Data size: 731652 bytes
Calculated num_elements: 60971
NetCDF file 'pillbox-rtop.ncdf' created successfully.
